In [1]:
import requests as req
import pandas as pd
import requests
import time
import re
from bs4 import BeautifulSoup as bsp
from selenium import webdriver
from seleniumwire import webdriver  # blinker == 1.7.0
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from fake_useragent import UserAgent
# from googlesearch import search
from datetime import datetime
from collections import defaultdict
basis = 'https://www.amazon.com/gp/new-releases'
base = 'https://www.amazon.com'
thirdWheel = '/s?k='
number_pattern = r"\d+"
maxDate = "June 1, 2023"
maxDateT = datetime.strptime(maxDate, "%B %d, %Y")
date_pattern = r"\b(?:January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}\b"

New Releases Categories

In [ ]:
cats = {}

options = {
    'headers' : {
        "User-Agent": UserAgent().random
    }
}
print("VISITING : ", basis)
driver = webdriver.Chrome(seleniumwire_options = options)
# driver = webdriver.Firefox()
driver.get(basis)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
driver.execute_script("window.scrollTo(0, 0);")
time.sleep(10)
html = driver.page_source
driver.quit()
soup = bsp(html, 'lxml')
board = soup.find('div', class_ = '_p13n-zg-nav-tree-all_style_zg-browse-group__88fbz')
moi = board.find_all('a')
for haha in moi:
    try:
        cats[haha.text] = base + haha['href']
    except:
        pass
print("CATEGORIES : ", cats)
df = pd.DataFrame(list(cats.items()), columns=['Category', 'Link'])
# Save the DataFrame to a CSV file
df.to_csv("NewCategory.csv", index=False)

Extracting More from New Releases

In [ ]:
# _p13n-zg-nav-tree-all_style_zg-browse-group__88fbz
df = pd.read_csv("NewCategory.csv")

    # Extract the link from the second column and add the array to the third column
for _, row in df.iterrows():
    link = row[1]  # Assuming the second column contains the link
    options = {
    'headers' : {
        "User-Agent": UserAgent().random
        }
    }
    print("VISITING : ", link)
    driver = webdriver.Chrome(seleniumwire_options = options)
    # driver = webdriver.Firefox()
    driver.get(link)
    time.sleep(10)

    # while True:
    #     # It will scroll to right above the footer of the page then scoll to the top then back to the bottom untill there is no new items being loaded
    #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #     time.sleep(5)
    #     driver.execute_script("window.scrollTo(0, 0);")
    #     time.sleep(5)
    #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #     time.sleep(5)
    #     New_Height = driver.execute_script("return document.body.scrollHeight")
    #     if New_Height == Previous_Height:
    #         break
    #     Previous_Height = New_Height
    #     # if there is a popup press the close button
    #     try:
    #         driver.find_element_by_class_name('modal-close').click()
    #     except:
    #         pass

    html = driver.page_source
    driver.quit()
    soup = bsp(html, 'lxml')
    board = soup.find('div', class_ = '_p13n-zg-nav-tree-all_style_zg-browse-group__88fbz', role = 'group')
    linkers = board.find_all('a')
    links = []
    for haha in linkers:
        try:
            links.append(base + haha['href'])
        except:
            pass
    print("LINKS : ", links)
    # Add the links to the third column of the DataFrame
    df.at[_, 'Links'] = ', '.join(links)
df.to_csv("NewCategory.csv", index=False)

In [3]:
df = pd.read_csv("NewCategory.csv", skiprows = 5)
csv_file = "D:\Products.csv"
first = False
# Extract the link from the second column and add the array to the third column
for _, row in df.iterrows():
    category = row[0]  # Assuming the first column contains the category name
    productLinks = row[2]  # Assuming the second column contains the link
    # print("PRODUCT LINKS : ", productLinks)

    for link in productLinks.split(', '):
        nextPage = link
        while nextPage:
            print("VISITING : ", nextPage)
            options = {
                'headers' : {
                    "User-Agent": UserAgent().random
                    }
            }
            driver = webdriver.Chrome(seleniumwire_options = options)
            # driver = webdriver.Firefox()
            driver.get(nextPage)
            Previous_Height = driver.execute_script("return document.body.scrollHeight")
            while True:
                # It will scroll to right above the footer of the page then scoll to the top then back to the bottom untill there is no new items being loaded
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1000);")
                time.sleep(5)
                driver.execute_script("window.scrollTo(0, 0);")
                time.sleep(5)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1000);")
                time.sleep(5)
                New_Height = driver.execute_script("return document.body.scrollHeight")
                if New_Height == Previous_Height:
                    break
                Previous_Height = New_Height
                # if there is a popup press the close button
                try:
                    driver.find_element_by_class_name('modal-close').click()
                except:
                    pass

            html = driver.page_source
            driver.quit()
            soup = bsp(html, 'lxml')
            board = soup.find('div', class_ = 'p13n-gridRow _cDEzb_grid-row_3Cywl')
            if board:
                products = board.find_all('div', class_ = 'p13n-sc-uncoverable-faceout')
                linkers = [product.find('a') for product in products]
                links = []
                for haha in linkers:
                    try:
                        links.append(base + haha['href'])
                    except:
                        pass
                print("LINKS : ", links)
                info = {"Categories": category, "Products": links}
                new_df = pd.DataFrame(info) 
                # if not first:
                #     new_df.to_csv(csv_file, mode='a', index = False)
                #     first = True
                # else:
                new_df.to_csv(csv_file, mode='a', header=False, index = False)
                print("|||||||||||||||||||||||||||| Data appended successfully with continued index. |||||||||||||||||||||||||||")
            nextPage = soup.find('li', class_ = 'a-last')
            if nextPage:
                try:
                    nextPage = base + nextPage.find('a')['href']
                    print("NEXT PAGE : ", nextPage)
                except:
                    nextPage = None
            else:
                print("NO NEXT PAGE")
                nextPage = None

C:\Users\Iqra Iqbal\AppData\Local\Temp\ipykernel_6120\1515872721.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  category = row[0]  # Assuming the first column contains the category name
C:\Users\Iqra Iqbal\AppData\Local\Temp\ipykernel_6120\1515872721.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  productLinks = row[2]  # Assuming the second column contains the link


VISITING :  https://www.amazon.com/gp/new-releases/audible/18571910011/ref=zg_bsnr_nav_audible_1
LINKS :  ['https://www.amazon.com/Audible-Studios-on-Brilliance-Heartbreaker/dp/B0DLH8GNKN/ref=zg_bsnr_g_18571910011_d_sccl_1/142-2989408-5028254?isALC=true', 'https://www.amazon.com/Reckoning-Part-Dramatized-Adaptation-Academy/dp/B0DT1MGBLC/ref=zg_bsnr_g_18571910011_d_sccl_2/142-2989408-5028254?isALC=true', 'https://www.amazon.com/Kingdom-Flesh-Fire-Dramatized-Adaptation/dp/B0DT1MC27N/ref=zg_bsnr_g_18571910011_d_sccl_3/142-2989408-5028254?isALC=true', 'https://www.amazon.com/Reckoning-Part-Dramatized-Adaptation-Academy/dp/B0DJMZ8ZYG/ref=zg_bsnr_g_18571910011_d_sccl_4/142-2989408-5028254?isALC=true', 'https://www.amazon.com/Lorne-Man-Invented-Saturday-Night/dp/B09HSV4RGL/ref=zg_bsnr_g_18571910011_d_sccl_5/142-2989408-5028254?isALC=true', 'https://www.amazon.com/Fabulous-Disaster-Garage-Madison-Square/dp/B0DD8T58W2/ref=zg_bsnr_g_18571910011_d_sccl_6/142-2989408-5028254?isALC=true', 'https://

KeyboardInterrupt: 

In [ ]:
csv_file = "D:\Products.csv"
productLinks = ["https://www.amazon.com/gp/new-releases/mobile-apps/9408874011/ref=zg_bsnr_nav_mobile-apps_1","https://www.amazon.com/gp/new-releases/mobile-apps/9408449011/ref=zg_bsnr_nav_mobile-apps_1","https://www.amazon.com/gp/new-releases/mobile-apps/9408491011/ref=zg_bsnr_nav_mobile-apps_1","https://www.amazon.com/gp/new-releases/mobile-apps/9408875011/ref=zg_bsnr_nav_mobile-apps_1","https://www.amazon.com/gp/new-releases/mobile-apps/9408464011/ref=zg_bsnr_nav_mobile-apps_1","https://www.amazon.com/gp/new-releases/mobile-apps/9408876011/ref=zg_bsnr_nav_mobile-apps_1","https://www.amazon.com/gp/new-releases/mobile-apps/10298308011/ref=zg_bsnr_nav_mobile-apps_1","https://www.amazon.com/gp/new-releases/mobile-apps/9408785011/ref=zg_bsnr_nav_mobile-apps_1","https://www.amazon.com/gp/new-releases/mobile-apps/9408914011/ref=zg_bsnr_nav_mobile-apps_1", "https://www.amazon.com/gp/new-releases/mobile-apps/9408850011/ref=zg_bsnr_nav_mobile-apps_1"]
for link in productLinks:
    nextPage = link
    while nextPage:
        print("VISITING : ", nextPage)
        options = {
        'headers' : {
            "User-Agent": UserAgent().random
            }
        }
        driver = webdriver.Chrome(seleniumwire_options = options)
        # driver = webdriver.Firefox()
        driver.get(nextPage)
        Previous_Height = driver.execute_script("return document.body.scrollHeight")
        while True:
            # It will scroll to right above the footer of the page then scoll to the top then back to the bottom untill there is no new items being loaded
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1000);")
            time.sleep(5)
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1000);")
            time.sleep(5)
            New_Height = driver.execute_script("return document.body.scrollHeight")
            if New_Height == Previous_Height:
                break
            Previous_Height = New_Height
            # if there is a popup press the close button
            try:
                driver.find_element_by_class_name('modal-close').click()
            except:
                pass
        html = driver.page_source
        driver.quit()
        soup = bsp(html, 'lxml')
        board = soup.find('div', class_ = 'p13n-gridRow _cDEzb_grid-row_3Cywl')
        if board:
            products = board.find_all('div', class_ = 'p13n-sc-uncoverable-faceout')
            linkers = [product.find('a') for product in products]
            links = []
            for haha in linkers:
                try:
                    links.append(base + haha['href'])
                except:
                    pass
            print("LINKS : ", links)
            info = {"Categories": category, "Products": links}
            new_df = pd.DataFrame(info) 
            new_df.to_csv(csv_file, mode='a', header=False, index = False)
            print("|||||||||||||||||||||||||||| Data appended successfully with continued index. |||||||||||||||||||||||||||")
        nextPage = soup.find('li', class_ = 'a-last')
        if nextPage:
            try:
                nextPage = base + nextPage.find('a')['href']
                print("NEXT PAGE : ", nextPage)
            except:
                nextPage = None
        else:
            print("NO NEXT PAGE")
            nextPage = None

In [ ]:
import pandas as pd

# Read the CSV file, skipping the header
file_path = "AmazonCategory.csv"
df = pd.read_csv(file_path, delimiter=";", header=None, skiprows=1)

# Create the dictionary with subcategories as keys and categories as values
category = {}

for _, row in df.iterrows():
    dawcategory, subcategory = row[0], row[1]
    
    if pd.notna(subcategory):  # Ignore empty subcategories
        category[subcategory] = dawcategory

# Print the dictionary
for subcategory, dawcategory in category.items():
    print(f"Subcategory: {subcategory} → Category: {dawcategory}")


In [ ]:
# session = requests.Session()
# session.headers.update({"User-Agent": UserAgent().random})
driver = webdriver.Chrome()
# driver = webdriver.Firefox(service = service, options = options)
driver.get(base)
#   It will scroll to right above the footer of the page then scoll to the top then back to the bottom untill there is no new items being loaded
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(10) 
html = driver.page_source
driver.quit()          
soup = bsp(html, 'lxml')
boxes = soup.find_all('a', class_ = 'a-link-normal _fluid-quad-image-label-v2_style_centerImage__30wh- aok-block image-window')
print(boxes)
links = []
urls = []
categ = []
for box in boxes:
    actual = box.get('href')
    url = base + actual
    print("Url : ", url)
    urls.append(url)
    som = box.get('aria-label')
    link = som.replace(' ', '+')
    link = base + thirdWheel + link
    print("Links : ", link)
    links.append(link)
    categ.append(som)
    print(som)


In [ ]:
urls = {}
for cat in category:
    url = base+thirdWheel+cat
    urls[url] = category[cat]
print(urls)


In [ ]:
def get_valid_proxies():
    proxy_list_url = 'https://free-proxy-list.net/'
    response = requests.get(proxy_list_url)
    soup = bsp(response.text, 'html.parser')
    proxy_data = []
    rows = soup.find_all('tr')[1:]
    for row in rows:
        columns = row.find_all('td')
        if len(columns) >= 8:
            ip_address = columns[0].text.strip()
            google_enabled = columns[5].text.strip().lower() == 'yes'
            https_enabled = columns[6].text.strip().lower() == 'yes'
            last_checked = columns[7].text.strip()
            if (last_checked.endswith('mins ago') and int(last_checked.split(' ')[0]) < 15) or last_checked.endswith('hours ago'):
                if google_enabled or https_enabled:
                    proxy_data.append({'ip_address': ip_address, 'google_enabled': google_enabled, 'https_enabled': https_enabled})

    return proxy_data


In [ ]:
def rotate_user_agent(proxy):
    if proxy:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'http': f'http://{proxy}',
            'https': f'https://{proxy}'
        }
    else:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    return headers

In [ ]:
products = []
# categories = []
cat = 0
me = False
# proxies = get_valid_proxies()

for url, cat in urls.items():
    total_samples = 0
    myurl = url
    while myurl:
        # proxy = proxies[total_samples % len(proxies)] if proxies else None
        options = {
            'headers' : {
                "User-Agent": UserAgent().random
            }
        }
        print("VISITING : ", myurl)
        # sdsd = Options()
        # sdsd.add_argument("--headless")  # Run in background
        # sdsd.add_argument("--disable-gpu")
        # sdsd.add_argument("--no-sandbox")
        # sdsd.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(seleniumwire_options = options)
        # driver = webdriver.Firefox()
        driver.get(myurl)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(10)
        html = driver.page_source
        driver.quit()
        # headers = rotate_user_agent(proxy)
        # response = req.get(myurl, headers = headers)
        # session = requests.Session()
        # session.headers.update({"User-Agent": UserAgent().random})
        # response = req.get(myurl, headers = {"User-Agent": UserAgent().random})
        # if response.status_code == 200:
            # soup = bsp(response.text, 'lxml')
        soup = bsp(html, 'lxml')
        # print(soup)
        productes = soup.find_all('a', class_ = 'a-link-normal s-no-outline')
        if not productes:
            productes = soup.find_all('a', class_ = 'a-link-normal s-line-clamp-4 s-link-style a-text-normal')
        if not productes:
            productes = soup.find_all('a' , class_= 'a-link-normal s-line-clamp-2 s-link-style a-text-normal')
        print(productes)
        for product in productes:
            product = base + product.get('href')
            products.append(product)
            print(product)
        csv_file = 'Amazon.csv'
        cat = cat.replace('+', " ")
        info = {"Products": products, "Categories": cat}
        new_df = pd.DataFrame(info)
        if not me:
            new_df.to_csv(csv_file, mode='a', index = False)
            me = True
        else:
            new_df.to_csv(csv_file, mode='a', header=False, index = False)
        print("|||||||||||||||||||||||||||| Data appended successfully with continued index. |||||||||||||||||||||||||||")
        
        total_samples += 1
        # Reset lists to prevent duplicates in the next iteration
        products.clear()
        nextP = soup.find('a', class_ = 's-pagination-item s-pagination-next s-pagination-button s-pagination-button-accessibility s-pagination-separator')
        if nextP:
            myurl = base + nextP.get('href')
        else:
            break
    cat += 1



In [5]:
# a-size-base prodDetAttrValue
# a-keyvalue a-vertical-stripes a-span6

csv_file = 'Products.csv'
meta = 'MetaData.csv'
rev = 'Reviews.csv'
df = pd.read_csv(csv_file)
samples = 0
me = False
for _,row in df.iterrows():
    escape = False
    options = {
    'headers' : {
        "User-Agent": UserAgent().random
        }
    }


    agent = {
        "main_category" : "",
        "title" : "",
        "average_rating" : 0.0,
        "rating_number" : 0,
        "features":[],
        "description":[],
        "price" : "",
        "images":[],
        "videos":[],
        "store" : "",
        "categories" : "",
        "details":{},
        "parent_asin" : "",
        "bought_together": [],
        "link" : "",
        "date" : ""
    }

    comment = {
        "rating" : 0.0,
        "title" : "",
        "text" : "",
        "images" : "",
        "asin" : "",
        "parent_asin" : "",
        "user_id" : "",
        "timestamp" : "",
        "verified_purchase" : 0,
        "helpful_vote" : "",
        "link" : ""
    }

    print("\n\n\n\n\n\n-------------------------------------------------------------------Product-------------------------------------------------------------------")

    # Extracting Links
    link = row['Products']
    print(link)
    agent["link"] = link

    # Extracting Categories
    category = row['Categories']
    print("Category :", category)
    agent["main_category"] = category


    driver = webdriver.Chrome(seleniumwire_options = options)
    driver.get(link)
    time.sleep(10)
    Previous_Height = driver.execute_script("return document.body.scrollHeight")
    print(f"Height : {Previous_Height}" )
    while True:
        # It will scroll to right above the footer of the page then scoll to the top then back to the bottom untill there is no new items being loaded
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1500);")
        time.sleep(5)
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1500);")
        time.sleep(5)
        New_Height = driver.execute_script("return document.body.scrollHeight")
        if New_Height == Previous_Height:
            break
        Previous_Height = New_Height
        # if there is a popup press the close button
        try:
            driver.find_element_by_class_name('modal-close').click()
        except:
            pass

    html = driver.page_source
    driver.quit()
    soup = bsp(html, 'lxml')

    # extracting details
    try:
        detailsBoard = soup.find('div', class_ = 'a-expander-content a-expander-section-content a-section-expander-inner').find('table', class_ = 'a-keyvalue prodDetTable').find_all('tr')
    except:
        try:
            detailsBoard = soup.find('div', id = 'detailBullets_feature_div').find_all('li')
        except:
            try:
                detailsBoard = soup.find('table', id = 'productDetails_detailBullets_sections1').find_all('tr')
            except:   
                try:
                    detailsBoard = soup.find('ul', class_ = 'a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list').find_all('tr')
                except:
                    detailsBoard = []
    details = {}
    if detailsBoard:
        for detailBoard in detailsBoard:
            detailingH = detailBoard.find('th').text.strip() if detailBoard.find('th') else None
            detailingA = detailBoard.find('td').text.strip() if detailBoard.find('td') else None
            if detailingH and detailingA:
                detailed = {detailingH: detailingA} 
                details |= detailed  
    else:
        details = {}
    print("Details :", details)
    agent["details"] = details

    date = ""
    try:
        for value in reversed(list(details.values())):
            if isinstance(value, str):
                date = re.search(date_pattern, value)
                print("Date Found", date)
                if date:
                    date = date.group(0)
                    dateT = datetime.strptime(date, "%B %d, %Y")
                    if dateT > maxDateT:
                        print(f"{date} is after {maxDateT}")
                    elif dateT < maxDateT:
                        escape = True
                    else:
                        print(f"{date} is the same as {maxDateT}")
                    break
    except:
        print("Date not found")
        date = ""
    print("Date :", date)
    agent["date"] = date
    
    if escape:
        print("PRODUCT SKIPPED AS IT WAS OLDER THAN MAX DATE")
        continue

    # extracting Title
    try:
        titleBoard = soup.find('span', id = 'productTitle')
        title = titleBoard.text.strip()
    except:
        title = ""
    print("Title :", title)
    agent["title"] = title

    # extracting ratings
    try:
        ratingBoard = soup.find('div', id = 'averageCustomerReviews')
        # extracting average rating
        avgRatingBoard = ratingBoard.find('span', class_ = 'a-size-base a-color-base')
        avgRating = avgRatingBoard.text.strip()
        # extracting rating number
        ratingNoBoard = ratingBoard.find('span', id = 'acrCustomerReviewText')
        ratingNo = ratingNoBoard.text.strip()
        junk = " ratings"
        ratingNo = ratingNo.replace(junk, "")
    except:
        avgRating = ""
        ratingNo = ""
    print("Average Rating :", avgRating)
    agent["average_rating"] = avgRating
    print("Rating :", ratingNo)
    agent["rating_number"] = ratingNo

    # extracting price
    try:
        priceBoard = soup.find('span', class_ = 'a-offscreen')
        if not priceBoard:
            # priceBoard = soup.find('div', class_ = 'a-section a-spacing-none aok-align-center aok-relative').find('span', class_ = 'aok-offscreen')
            priceBoard = soup.find('span', class_ = 'a-price-whole')
            priceBoard2 = soup.find('span', class_ = 'a-price-fraction')
            price2 = priceBoard2.text.strip()
            price = priceBoard.text.strip()
            price = price + price2
        else:
            price = priceBoard.text.strip()
    except:
        price = ""
    print("Price :", price)
    agent["price"] = price

    # exracting media
    try:
        mediaBoard = soup.find('div', id = 'altImages')
        if not mediaBoard:
            mediaBoard = soup.find('ul', class_ = 'a-unordered-list a-nostyle a-button-list a-vertical a-spacing-top-micro regularAltImageViewLayout')
            if not mediaBoard:
                mediaBoard = soup.find('ul', class_ = 'a-unordered-list a-nostyle a-button-list a-vertical a-spacing-top-micro gridAltImageViewLayoutIn1x7')
                if not mediaBoard:
                    mediaBoard = soup.find('ul', class_ = 'a-unordered-list a-nostyle a-button-list a-vertical a-spacing-top-extra-large regularAltImageViewLayout')
                    if not mediaBoard:
                        mediaBoard = soup.find('ul', class_ = 'a-unordered-list a-nostyle a-button-list a-declarative a-button-toggle-group a-vertical a-spacing-top-extra-large regularAltImageViewLayout')
        media = mediaBoard.find_all('img')
        images = [img['src'] for img in media]
        videos = [images[-1]]
        images = images[0:-1]
    except:
        images = []
        vid = []
    print("Images :", images)
    agent["images"] = images   
    print("Videos :", videos)
    agent["videos"] = videos

    # extracting the store
    try:
        storeBoard = soup.find('tr', class_ = 'a-spacing-small po-brand').find('td', class_ = 'a-span9')
        store = storeBoard.text.strip()
    except:
        try:
            storeBoard = soup.find('a', id = 'bylineInfo').text.strip()
            junk = "Visit the "
            store = storeBoard.replace(junk, "")
        except:
            store = ""
    print("Store :", store)
    agent["store"] = store

    # extracting features
    try:
        featuresBoard = soup.find('ul', class_ = 'a-unordered-list a-vertical a-spacing-mini').find_all('li')
        features = [feature.text.strip() for feature in featuresBoard]
    except:
        features = []
    print("Features :", features)
    agent["features"] = features
        
                    
    # extracting asin
    try:
        asin = details['ASIN']
    except:
        asin = ""
    print("ASIN :", asin)
    agent["parent_asin"] = asin

    # agent["date"] = date

    # extracting description
    try:
        descriptionBoard = soup.find('table', class_ = 'a-normal a-spacing-micro')
        if not descriptionBoard:
            descriptionBoard = soup.find('ul', id = 'a-nostyle').find_all('div', class_ = 'a-fixed-left-grid-col a-col-right')
        descriptions = descriptionBoard.find_all('td', class_ = 'a-span9')
        description = [dec.text.strip() for dec in descriptions]
    except:
        description = []
    print("Description :", description)
    agent["description"] = description

    # extracting bought together
    try:
        boughtTogetherBoard = soup.find('div', id = 'a-cardui _c3AtZ_new-thumbnail-box_1W9Ku _c3AtZ_two-item-thumbnail-box_7kF95')
        if not boughtTogetherBoard:
            boughtTogetherBoard = soup.find('div', class_ = 'a-cardui _p13n-desktop-sims-fbt_fbt-desktop_new-thumbnail-box__36bD3 _p13n-desktop-sims-fbt_fbt-desktop_two-item-thumbnail-box__jV2am')
        if not boughtTogetherBoard:
            boughtTogetherBoard = soup.find('div', class_ = 'a-cardui _p13n-desktop-sims-fbt_fbt-desktop_new-thumbnail-box__36bD3')
        boughtTogetherS = boughtTogetherBoard.find_all('a', class_ = 'a-link-normal a-text-normal', href = True)
        if not boughtTogetherS:
            boughtTogetherS = boughtTogetherBoard.find_all('a', class_ = 'a-link-normal', href = True)        
        boughtTogether = [base + link['href'] for link in boughtTogetherS]
    except:
        boughtTogether = []
    print("Bought Together :", boughtTogether)
    agent["bought_together"] = boughtTogether

    # extracting further categories
    try:
        categoriesBoard = soup.find_all('div', id= 'tp-inline-twister-dim-values-container')
        categoriesBoardAh = categoriesBoard[-1].find_all('li')
        categories = [categoriesBoardh.text.strip() for categoriesBoardh in categoriesBoardAh]
    except:
        categories = []
    print("Categories :", categories)
    agent["categories"] = categories


    ############################################################################################################################################################
    print("\n\n\n-------------------------------------------------------------------Comments-------------------------------------------------------------------")
    # extracting reviews
    try:
        reviewsUS = soup.find('ul', id = 'cm-cr-dp-review-list').find_all('li')
    except:
        reviewUS = []

    try:
        reviewsGlobal = soup.find('ul', id = 'cm-cr-global-review-list').find_all('li')
    except:
        reviewsGlobal = []
    reviews = reviewsUS + reviewsGlobal
    for review in reviews:
        try:
            # extracting rating
            reviewRatingBoard = review.find('h5')
            reviewRatingBoard = reviewRatingBoard.find_all('span')
            try:
                reviewRating = reviewRatingBoard[0].text.strip()
                print("Rating Found")
                reviewRating = reviewRating[0:3]
            except:
                reviewRating = ""

            try:
                for span in reversed(reviewRatingBoard):  # iterate from the end
                    text = span.get_text(strip=True)
                    if text:  # if text is not empty
                        reviewTitle = text
                        break
                print("Title Found")
            except:
                reviewTitle = ""
        except:
            reviewRating = ""
            reviewTitle = ""
        print("Rating :", reviewRating)
        comment["rating"] = reviewRating

        # extracting title
        print("Title :", reviewTitle)
        comment["title"] = reviewTitle

        # extracting Text
        try:
            reviewTextBoard = review.find('div', class_ = 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content')
            reviewText = reviewTextBoard.text.strip()
        except:
            reviewText = ""
        print("Review :", reviewText)
        comment["text"] = reviewText

        # extracting images
        try:
            reviewImages = review.find('div', class_ = 'review-image-tile-section').find_all('img')
            Images = [img['src'] for img in reviewImages]
        except:
            Images = []
        print("Images :", Images)
        comment["images"] = Images

        # extracting ASIN
        try:
            reviewAsin = review.get('id')
        except:
            reviewAsin = ""
        print("ASIN :", reviewAsin)
        comment["asin"] = reviewAsin

        # extracting parent ASIN
        reviewParentAsin = asin
        print("Parent ASIN :", reviewParentAsin)
        comment["parent_asin"] = reviewParentAsin

        # eaxtracting username
        try:
            username = review.find('span', class_ = 'a-profile-name').text.strip()
        except:
            username = ""
        print("Username :", username)
        comment["user_id"] = username

        # extracting date
        try:
            reviewTimeBoard = review.find('span', class_ = 'a-size-base a-color-secondary review-date')
            reviewTimes = reviewTimeBoard.text
            reviewTime = re.search(date_pattern, reviewTimes)
            reviewTime = reviewTime.group(0)
            print("Date Found")
        except:
            reviewTime = ""
        print("Date :", reviewTime)
        comment["timestamp"] = reviewTime
        verified = False
        if review.find('span', class_ = 'a-size-mini a-color-state a-text-bold'):
            verified = True
        print("Verified :", verified)
        comment["verified_purchase"] = verified

        # extracting helpfulness
        try:
            reviewHelpful = review.find('span', class_ = 'a-size-base a-color-tertiary cr-vote-text')
            reviewHelpful = reviewHelpful.text
            haha = "One"
            helpful = 0
            Helpfulness = re.search(number_pattern, reviewHelpful)
            if Helpfulness:
                helpful = int(Helpfulness.group())
            elif haha in reviewHelpful:
                helpful = 1
        except:
            helpful = 0
        print("Helpful :", helpful)
        comment["helpful_vote"] = helpful

        # extracting link
        comment["link"] = link
        myMeta = pd.DataFrame([agent])
        myRev = pd.DataFrame([comment])
    if not me:
        myMeta.to_csv(meta, mode='a', index = False)
        myRev.to_csv(rev, mode='a', index = False)
        me = True
    else:
        myMeta.to_csv(meta, mode='a', header=False, index = False)
        myRev.to_csv(rev, mode='a', header=False, index = False)

    print("|||||||||||||||||||||||||||| Data appended successfully with continued index. |||||||||||||||||||||||||||")







-------------------------------------------------------------------Product-------------------------------------------------------------------
https://www.amazon.com/Amazon-Paperwhite-Lightweight-Water-Safe-Protective/dp/B0CM7X424T/ref=zg_bsnr_g_370783011_d_sccl_1/134-8633991-4215122?psc=1
Category : Amazon Devices & Accessories
Height : 8541
Details : {}
Date : 
Title : All-new Amazon Kindle Paperwhite and Kindle Colorsoft Signature Edition Case, Lightweight and Water-Safe, Foldable Protective Cover - Fabric
Average Rating : 4.7
Rating : 3,292
Price : Page 1 of 1
Images : ['https://m.media-amazon.com/images/G/01/HomeCustomProduct/360_icon_73x73v2._CB485971279_SS40_FMpng_RI_.png', 'https://m.media-amazon.com/images/I/41Pf4gddh4L._AC_US40_.jpg', 'https://m.media-amazon.com/images/I/41kN0EvrBBL._AC_US40_.jpg', 'https://m.media-amazon.com/images/I/31FR03uprzL._AC_US40_.jpg', 'https://m.media-amazon.com/images/I/41nMtaxVULL._AC_US40_.jpg', 'https://m.media-amazon.com/images/I/31HzXmHh

KeyboardInterrupt: 

In [ ]:


# Define the date strings
date_str1 = "June 22, 2016"
date_str2 = "September 30, 2023"

# Convert the date strings to datetime objects
date1 = datetime.strptime(date_str1, "%B %d, %Y")
date2 = datetime.strptime(date_str2, "%B %d, %Y")

# Compare the dates
if date1 > date2:
    print(f"{date_str1} is greater than {date_str2}")
else:
    print(f"{date_str1} is not greater than {date_str2}")

In [ ]:
# Date string in the format "Month Day, Year"
date_str = "January 17, 2025"

# Parse the string into a datetime object
date_obj = datetime.strptime(date_str, "%B %d, %Y")

# Convert to Unix time
unix_time = int(date_obj.timestamp())

print(f"Unix Time: {unix_time}")



In [ ]:
# Example statement
statement = "The item costs 700 dollars, and the discount is."

# Regular expression to match one or more digits
number_pattern = r"\d+"

# Find the first number in the statement
match = re.search(number_pattern, statement)

if match:
    number = int(match.group())  # Convert the matched number to an integer
    print(f"Extracted number: {number}")
else:
    print("No number found.")
